## Setup libraries

In [ ]:
!pip uninstall pycocotools -y
!pip install -q git+https://github.com/waleedka/coco.git#subdirectory=PythonAPI

In [ ]:
!pip install hydra-core
!pip install pytorch-lightning==0.8.1

## Import libraries

In [4]:
import os
import warnings

import hydra
from hydra.experimental import initialize, compose
from hydra.core.global_hydra import GlobalHydra
import pytorch_lightning as pl
import torch
from omegaconf import DictConfig
from pytorch_lightning.loggers import TensorBoardLogger, CometLogger
from pytorch_lightning import seed_everything

from src.lightning_classes.lightning_wheat import LitWheat
from src.utils.loggers import JsonLogger
from src.utils.utils import set_seed, save_useful_info, flatten_omegaconf

warnings.filterwarnings("ignore")

## Configure

In [9]:
GlobalHydra.instance().clear()
initialize(config_path="conf")
cfg=compose(config_name="config.yaml")
cfg.data.folder_path = "C:/Users/alberto/Downloads/global-wheat-detection"

# If internet is disabled
# cfg['model']['backbone']['params']['pretrained'] = False
# cfg['model']['backbone']['params']['pretrained_backbone'] = False

print(cfg.pretty())
# save_useful_info()

general:
  save_dir: logs/
  workspace: aburgos
  project_name: wheat
  device: cuda
dataset:
  class_name: src.utils.dataset.WheatDataset
trainer:
  gpus: 1
  distributed_backend: dp
  profiler: false
  max_epochs: 10
  log_save_interval: 100
  gradient_clip_val: 1
  num_sanity_val_steps: 0
  fast_dev_run: false
  limit_train_batches: 0.05
  limit_val_batches: 0.1
  deterministic: true
training:
  lr: 0.001
  metric: main_score
  mode: max
  seed: 666
  debug: false
logging:
  log: true
optimizer:
  class_name: torch.optim.SGD
  params:
    lr: ${training.lr}
    momentum: 0.9
    weight_decay: 0.0005
scheduler:
  class_name: torch.optim.lr_scheduler.ReduceLROnPlateau
  step: epoch
  monitor: ${training.metric}
  params:
    mode: ${training.mode}
    factor: 0.1
    patience: 3
model:
  backbone:
    class_name: torchvision.models.detection.fasterrcnn_resnet50_fpn
    params:
      pretrained: true
      num_classes: 91
      pretrained_backbone: true
  head:
    class_name: torchvis

In [15]:
seed_everything(cfg.training.seed)
hparams = flatten_omegaconf(cfg)
model = LitWheat(hparams=hparams, cfg=cfg)

early_stopping = pl.callbacks.EarlyStopping(**cfg.callbacks.early_stopping.params)
model_checkpoint = pl.callbacks.ModelCheckpoint(**cfg.callbacks.model_checkpoint.params)
lr_logger = pl.callbacks.LearningRateLogger()

tb_logger = TensorBoardLogger(save_dir=cfg.general.save_dir)
comet_logger = CometLogger(save_dir=cfg.general.save_dir,
                           workspace=cfg.general.workspace,
                           project_name=cfg.general.project_name,
                           api_key=cfg.private.comet_api,
                           experiment_name=os.getcwd().split('\\')[-1])
json_logger = JsonLogger()

trainer = pl.Trainer(
    logger=[tb_logger, comet_logger,
            json_logger],
    early_stop_callback=early_stopping,
    checkpoint_callback=model_checkpoint,
    callbacks=[lr_logger],
    **cfg.trainer,
)
trainer.fit(model)

# save as a simple torch model
model_name = os.getcwd().split('\\')[-1] + '.pth'
print(model_name)
torch.save(model.model.state_dict(), model_name)

CometLogger will be initialized in online mode
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/aburgos/wheat/eaefb40a5153484f82c946554f1b0731
COMET INFO:   Others:
COMET INFO:     Name : object_detector_template
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (151 KB)
COMET INFO:     installed packages       : 1
COMET INFO: ---------------------------
COMET INFO: old comet version (3.1.12) detected. current: 3.1.13 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/aburgos/wheat/5de7b7b92c424ba393f3965dcb15a66c

GPU available: True, used: T

creating index...
index created!
Epoch 1:   4%| | 2/45 [00:05<01:52,  2.62s/it, loss=4.010, v_num=0_5de7b7b92c424ba393f3965dcb15a66c_None, loss_classifi

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/aburgos/wheat/5de7b7b92c424ba393f3965dcb15a66c
COMET INFO:   Metrics:
COMET INFO:     lr-SGD : 0.001
COMET INFO:   Others:
COMET INFO:     Name : object_detector_template
COMET INFO:   Parameters:
COMET INFO:     augmentation_train_augs_0_params_p           : 1.0
COMET INFO:     augmentation_valid_augs_0_params_p           : 1.0
COMET INFO:     callbacks_early_stopping_params_patience     : 5
COMET INFO:     callbacks_model_checkpoint_params_save_top_k : 1
COMET INFO:     data_batch_size                              : 4
COMET INFO:     data_num_workers                             : 1
COMET INFO:     model_backbone_params_num_classes            : 91
COMET INFO:     model_head_params_num_classes                : 2
COMET INFO:     optimizer_params_mo

COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Waiting for completion of the file uploads (may take several seconds)
COMET INFO: Still uploading


object_detector_template.pth
